## Datasets Matching (Scopus - Google Scholar - Web Of Science)

In [11]:
import pandas as pd
import numpy as np
import unidecode

In [12]:
scop = pd.read_csv("datasets/scopus_dataset.csv")
scop = scop.drop_duplicates()
google_sch = pd.read_csv("datasets/scholar_dataset.csv")
wos = pd.read_csv("datasets/authors_data_wos.csv")

- Harmoniser les noms de colonnes pour simplifier le traitement

In [3]:
# Harmoniser les noms de colonnes pour `wos`
scop.rename(columns={
    "Author_ID": "Author_ID_scop",
    "Author_Name": "Author_Name_scop",
    "Author_Affiliation": "Author_Affiliation_scop",
    "Author_Citations": "Author_Citations_scop",
    "Author_h-index": "Author_h-index_scop",
    "Co_authors_Names": "Co_authors_Names_scop"
}, inplace=True)
wos.rename(columns={
    "ID de l'Auteur": "Author_ID_wos",
    "nom_complet": "Author_Name_wos",
    "pays_affiliation": "Author_Affiliation_wos",
    "Sum of Times Cited": "Author_Citations_wos",
    "H-index": "Author_h-index_wos",
    "co_auteurs": "Co_authors_Names_wos",
    'Articles' : 'Articles_wos' ,
    'Journal' : 'Journal_wos'
}, inplace=True)
google_sch.rename(columns={
    "ID de l'Auteur": "Author_ID_gosch",
    "Nom Complet": "Author_Name_gosch",
    "Pays d'Affiliation": "Author_Affiliation_gosch",
    "Citations Totales": "Author_Citations_gosch",
    "H-index": "Author_h-index_gosch",   
    "FWCI": "Author_FWCI_gosch",
    "Co-auteurs": "Co_authors_Names_gosch"
}, inplace=True)

In [13]:
print(scop.columns)
print(wos.columns)
print(google_sch.columns)

Index(['Author_ID', 'Author_Name', 'Author_Affiliation', 'Author_Citations',
       'Author_Documents', 'Author_h-index', 'Author_FWCI', 'Co_Authors_IDs',
       'Co_authors_Names'],
      dtype='object')
Index(['ID de l'Auteur', 'nom_complet', 'pays_affiliation', 'co_auteurs',
       'H-Index', 'Sum of Times Cited', 'Articles', 'Journal'],
      dtype='object')
Index(['ID de l'Auteur', 'Nom Complet', 'Pays d'Affiliation',
       'Citations Totales', 'H-index', 'FWCI', 'Co-auteurs'],
      dtype='object')


In [5]:
wos

,Author_ID_wos,Author_Name_wos,Author_Affiliation_wos,Co_authors_Names_wos,H-Index,Author_Citations_wos,Articles_wos,Journal_wos
0,29091850,"Aghriche, Ahmed",MOROCCO,[],2,26,[{'Titre de l’article': 'OSCILLATIONS INDUCED ...,"[{'issn': '1937-1632', 'scope': 'Series S of D..."
1,7137580,"Dargham, Abdelmajid",MOROCCO,[],1,2,[{'Titre de l’article': 'REDUCING THE INTERPRO...,"[{'issn': '1895-1767', 'scope': 'The area of s..."
2,20979755,"El Bannay, Omar",MOROCCO,[],2,27,[{'Titre de l’article': 'Determining the potat...,"[{'issn': '0957-0233', 'scope': 'Measurement ..."
3,20988767,"Fatmi, Nadia Idrissi",MOROCCO,[],1,4,[{'Titre de l’article': 'MATHEMATICAL MODEL AN...,"[{'issn': '2052-2541', 'scope': 'Information n..."
4,29836419,"Ifzarne, Aziz",MOROCCO,[],3,32,[{'Titre de l’article': 'Quarter-Tone Music: A...,"[{'issn': '0302-9743', 'scope': 'This distingu..."
5,63349824,"Lamghari, Nidal",MOROCCO,[],0,0,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
6,50018709,noureddine aboutabit,NaN,"['Lamghari, Nidal', 'Dahbali, Mohamed', 'Mait,...",1,1,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
7,63421194,"Dahbali, Mohamed",MOROCCO,[],0,0,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
8,63939275,"Mait, Hind Ait",MOROCCO,[],0,0,[{'Titre de l’article': 'Unsupervised VAD meth...,"[{'issn': '0952-8091', 'scope': 'IJCAT address..."
9,12149340,"Nasri, Mohamed",MOROCCO,[],2,11,[{'Titre de l’article': 'Semantic Analysis of ...,"[{'issn': None, 'scope': 'N/A', 'index': 'WOS'..."


In [6]:
scop["Author_Name_scop"][:1]

0    Imad, Hafidi
Name: Author_Name_scop, dtype: object

In [7]:
wos["Author_Name_wos"][:1]

0    Aghriche, Ahmed
Name: Author_Name_wos, dtype: object

In [8]:
google_sch["Author_Name_gosch"][:1]

0    Abdelghani Ghazdali
Name: Author_Name_gosch, dtype: object

**Remplir les valeurs manquantes.**

**Avec les règles:** 
- Priorité à scop.
- Utiliser wos si scop est vide.
- Si les deux sont nulles, insérer NaN.


- Effectuer la jointure entre scop, wos, et google scholar

In [9]:
import re
def name_to_word_set(name):
    if pd.isnull(name):
        return set()  # Gérer les valeurs manquantes
    # Supprimer les caractères spécifiques et normaliser
    cleaned_name = re.sub(r'[^\w\s]', '', name)  # Garder uniquement les lettres, chiffres et espaces
    normalized_name = unidecode.unidecode(cleaned_name.lower().strip())  # Supprimer les accents, passer en minuscule
    return set(normalized_name.split()) 

name_to_word_set("Imad, Hafidi")

{'hafidi', 'imad'}

In [10]:
name_to_word_set("imad hafidi"), name_to_word_set(" Hafidi, Imad"), name_to_word_set("imad hafidi")

({'hafidi', 'imad'}, {'hafidi', 'imad'}, {'hafidi', 'imad'})

In [ ]:
def find_matches(df1, col1, df2, col2):
    matches = []
    for idx1, name1 in enumerate(df1[col1]):
        set1 = name_to_word_set(name1)
        match_found = None
        for idx2, name2 in enumerate(df2[col2]):
            set2 = name_to_word_set(name2)
            if set1 == set2:
                match_found = df2.iloc[idx2].to_dict()
                print(f"Match found: {name1} ({set1}) == {name2} ({set2})")
                break
        # if match_found is None:
#             print(f"No match for: {name1} ({set1})")
        matches.append(match_found)
    return pd.DataFrame(matches)


# Jointure entre Scopus et WoS
scop_wos_matched = find_matches(scop, 'Author_Name_scop', wos, 'Author_Name_wos')

# Ajouter les résultats de WoS à Scopus
scop_wos_merged = pd.concat([scop, scop_wos_matched.add_suffix('_wos')], axis=1)

# Jointure entre le résultat précédent et Google Scholar
final_matched = find_matches(scop_wos_merged, 'Author_Name_scop', google_sch, 'Author_Name_gosch')

# Ajouter les résultats de Google Scholar
final_merged = pd.concat([scop_wos_merged, final_matched.add_suffix('_gosch')], axis=1)

In [ ]:
final_merged.head(50)

In [ ]:
final_merged.head(50)

In [ ]:
final_merged = final_merged.drop(columns=["0_wos", "0_gosch"])
final_merged.head(1)

In [ ]:
final_merged.isnull().sum()

In [ ]:
import re 

# Fonction pour extraire le nom du pays
def extract_country(affiliation):
    if pd.isnull(affiliation):  # Vérifie si la valeur est NaN
        return None
    match = re.search(r'-\s*(\w+)$', affiliation)  # Recherche le texte après le tiret
    return match.group(1) if match else affiliation  # Retourne le pays s'il est trouvé

# Appliquer la fonction sur la colonne 'Author_Affiliation_scop'
final_merged['Author_Affiliation_scop'] = final_merged['Author_Affiliation_scop'].apply(extract_country)

final_merged = final_merged.drop(columns=["Author_ID_wos", "Author_Name_wos", "Author_Affiliation_wos", "Co_authors_Names_wos", "H-Index","Author_Citations_wos","Articles", "Journal"])

# Afficher les premières lignes pour validation
final_merged["Author_Affiliation_scop"].head(2)

In [ ]:
final_merged.isnull().sum()

In [ ]:
try:
    final_merged.to_csv("datasets/merged_authors_datasets.csv", index=False)
    print("Dataset saved")
except: 
    print("Error..")

In [ ]:
df = pd.read_csv("datasets/merged_authors_datasets.csv")
df.head(3)

In [ ]:
df.rename(columns={
    "Author_ID_scop": "Author_ID",
    "Author_Name_scop": "Author_Name",
    "Author_Affiliation_scop" : "Author_Affiliation",
    "Author_Citations_scop": "Author_Citations",
    "Author_Documents": "Author_Documents_number",
    "Author_h-index_scop" : "H-index",
    "Author_FWCI" : "FWCI",
    "Co_authors_Names_scop" : "Co_authors_Names"
}, inplace=True)

In [ ]:
df.columns


In [ ]:
try:
    df.to_csv("datasets/merged_authors_datasets.csv", index=False)
    print("Dataset saved")
except: 
    print("Error..")

**Documents/Journals Matching**

In [ ]:
df1 = pd.read_csv("datasets/merged_authors_datasets.csv")
df2 = pd.read_json('datasets/scopus_docs_journals.json')
df3_articles = pd.read_csv("datasets/partial_article_info_gosch.csv")
df4_journals = pd.read_csv('datasets/partial_journal_info_gosch.csv')

In [ ]:
df1.head(1)

In [ ]:
df2.head(1)

In [ ]:
df3_articles.head(1)

In [ ]:
df4_journals.head(1)

In [ ]:
df3_articles.shape ,df4_journals.shape

In [ ]:
df3_articles.duplicated().sum(), df4_journals.duplicated().sum()

In [ ]:
print(df3_articles['ISSN'].nunique())  # Nombre d'ISSN uniques dans df3_articles
print(df4_journals['ISSN'].nunique())  # Nombre d'ISSN uniques dans df4_journals

In [ ]:
print(df3_articles['ISSN'].value_counts())  # Vérifie combien de fois chaque ISSN apparaît
print(df4_journals['ISSN'].value_counts())

In [ ]:
df3_articles_unique = df3_articles.drop_duplicates(subset=['ISSN'])
df4_journals_unique = df4_journals.drop_duplicates(subset=['ISSN'])

merged_df_gosch = pd.merge(df3_articles_unique, df4_journals_unique, on='ISSN', how='inner')
print(merged_df_gosch.shape)  # Cela devrait limiter le nombre de lignes


In [ ]:
merged_df_gosch.head(2)

In [ ]:
merged_df_gosch.duplicated().sum()

In [ ]:
merged_df_gosch.isna().sum()

In [ ]:
merged_df_gosch.columns = [col + "_gosch" for col in merged_df_gosch.columns]

- Les Articles de Scopus

In [ ]:
df2["articles"]

In [ ]:
df2["articles"][0]


In [ ]:
df2["articles"].isnull().sum()

**Résultat attendu : Chaque ligne représente un article avec ses informations détaillées.**

- title
- pub_year
- citations
- issn
- DOI
- document type
- source type
- abstract
- authors
- author_keywords
- Nom de la revue
- H-index
- Editeur
- journal_issn
- index
- Portee thematique
- Quartile.quartile_value
- Score SJR
- Impact Factor.impact_factor_value


In [ ]:
# Assuming df2 is the DataFrame containing the 'articles' column
data = df2["articles"]

processed_data = []

# Iterate over each list in the 'articles' column
for article_list in data:
    if isinstance(article_list, list):  # Ensure the entry is a list
        for article in article_list:  # Iterate over each dictionary (article)
            if isinstance(article, dict):  # Ensure the element is a dictionary
                row = {
                    "title": article.get("title"),
                    "pub_year": article.get("pub year"),
                    "citations": article.get("citations"),
                    "issn": article.get("issn"),
                    "DOI": article.get("DOI"),
                    "document_type": article.get("document type"),
                    "source_type": article.get("source type"),
                    "abstract": article.get("abstract"),
                    "authors": ", ".join(article.get("authors", [])),
                    # Ensure 'author keywords' is a list or empty list before joining
                    "author_keywords": ", ".join(article.get("author keywords", [])) if isinstance(article.get("author keywords", []), list) else ""
                }

                # Check if 'journal info' exists and is not None before processing
                journal_info = article.get("journal info")
                if journal_info:
                    row.update({
                        "journal_name": journal_info.get("name"),
                        "journal_publisher": journal_info.get("publisher"),
                        "journal_issn": journal_info.get("issn"),
                        "journal_scope": journal_info.get("scope"),
                        "journal_quartile": journal_info.get("quartile"),
                    })
                
                processed_data.append(row)
            else:
                print(f"Unexpected article type: {type(article)}, article: {article}")
    else:
        print(f"Unexpected entry type in data: {type(article_list)}, entry: {article_list}")

# Convert the processed data to a DataFrame for easier analysis
processed_df = pd.DataFrame(processed_data)

In [ ]:
processed_df.head(2)

- renommer les colonnes

In [ ]:
processed_df.columns = [col + "_scopus" for col in processed_df.columns]

In [ ]:
processed_df.isnull().sum()

In [ ]:
print(processed_df.shape, processed_df.duplicated().sum())
processed_df = processed_df.drop_duplicates()

In [ ]:
processed_df.duplicated().sum()

In [ ]:
processed_df.isna().sum()

In [ ]:
merged_df_gosch.isna().sum()


**Explications des mappages :**

- title_scopus ↔ Titre de l'article_gosch
- pub_year_scopus ↔ Année de publication_gosch
- citations_scopus ↔ Nombre de citations_gosch
- issn_scopus ↔ ISSN_gosch
- DOI_scopus ↔ DOI_gosch
- abstract_scopus ↔ Résumé_gosch
- journal_name_scopus ↔ Nom_gosch
- journal_publisher_scopus ↔ Editeur_gosch
- journal_issn_scopus ↔ ISSN_gosch
- journal_scope_scopus ↔ Portee thematique_gosch
- journal_quartile_scopus ↔ Quartile_gosch

In [ ]:
# Colonnes correspondantes à synchroniser entre les deux DataFrames
columns_to_sync = {
    "title_scopus" : "Titre de l'article_gosch",
    "pub_year_scopus" : "Année de publication_gosch",
    "citations_scopus" : "Nombre de citations_gosch",
    "issn_scopus" : "ISSN_gosch",
    "DOI_scopus" : "DOI_gosch",
    "abstract_scopus" : "Résumé_gosch",
    "journal_name_scopus" : "Nom_gosch",
    "journal_publisher_scopus" : "Editeur_gosch",
    "journal_issn_scopus" : "ISSN_gosch",
    "journal_scope_scopus" : "Portee thematique_gosch",
    "journal_quartile_scopus" : "Quartile_gosch"
}

In [ ]:
# Effectuer une jointure sur la colonne ISSN
merged_result = processed_df.merge(
    merged_df_gosch,
    left_on="issn_scopus",
    right_on="ISSN_gosch",
    how="outer"
)

In [ ]:
# Synchroniser les colonnes
for processed_col, gosch_col in columns_to_sync.items():
    merged_result[processed_col] = merged_result[processed_col].combine_first(merged_result[gosch_col])
    merged_result[gosch_col] = merged_result[gosch_col].combine_first(merged_result[processed_col])

In [ ]:
merged_result.head()

In [ ]:
# Diviser à nouveau en deux DataFrames avec les colonnes mises à jour
updated_processed_df = merged_result[[col for col in processed_df.columns]]
updated_merged_df_gosch = merged_result[[col for col in merged_df_gosch.columns]]

In [ ]:
updated_processed_df.isnull().sum()

In [ ]:
processed_df["title_scopus"]
title_scopus, document_type_scopus

Fin.